# Importing Dataset from gdrive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
ZIPPED_PATH="/content/drive/MyDrive/car/car.zip"

In [ ]:
import torch
import torchvision.transforms as T
import torchvision.models as models
from PIL import Image
import cv2
import zipfile
import numpy as np
import pandas as pd
import os
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Destination folder for extraction
extract_folder = "/content/cars"

# Create the destination folder if it doesn't exist
if not os.path.exists(extract_folder):
    os.makedirs(extract_folder)

with zipfile.ZipFile(ZIPPED_PATH, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)
print("Extraction complete.")

# Load pre-trained ResNet-50 model (without final FC layer)
resnet = models.resnet50(pretrained=True)
resnet.fc = torch.nn.Identity()  # Remove final layer to get feature vector
resnet.to(device)  # Move model to device
resnet.eval()

# Define Image Transform
transform = T.Compose([
    T.Resize((224, 224)),  # Resize for ResNet input
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])


# Function to extract features
def extract_features(img_path):
  try:
    image = Image.open(img_path).convert("RGB") # Ensure RGB mode
  except Exception as e:
    print("Error opening image:", e)

  image = transform(image).unsqueeze(0).to(device) # Apply transforms & move to GPU/CPU
  with torch.no_grad():
      feature_vector = resnet(image).cpu().numpy().flatten()  # Extract features
  return feature_vector

# Load annotations
trainDF = pd.read_excel('/content/drive/MyDrive/car/cars_with_class_names.xlsx', sheet_name='train')
testDF = pd.read_excel('/content/drive/MyDrive/car/cars_with_class_names.xlsx', sheet_name='test')

# Convert DataFrame to NumPy Array
trainArray = trainDF.to_numpy()
testArray = testDF.to_numpy()

# Paths to images
train_folder = os.path.join(extract_folder, "cars_train", "cars_train")
test_folder = os.path.join(extract_folder, "cars_test", "cars_test")

#train_folder = "/content/drive/MyDrive/car/cars_train/cars_train/"
#test_folder = "/content/drive/MyDrive/car/cars_test/cars_test/"

# Extract train features and labels
X_train, y_train = [], []
for entry in trainArray:
    img_path = os.path.join(train_folder, entry[7])  # Assuming filename is in first column
    if os.path.exists(img_path):
        X_train.append(extract_features(img_path))  # Extract features
        y_train.append(entry[5])  # Assuming class label is in second column

# Extract test features and labels
X_test, y_test = [], []
for entry in testArray:
    filename = str(entry[6])
    filename_fixed = filename.strip("'\"")
    img_path = os.path.join(test_folder, filename_fixed)
    if os.path.exists(img_path):
        X_test.append(extract_features(img_path)) # Extract features
        y_test.append(entry[4])  # Assuming class label is in second column


# Convert to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

# Encode labels into numeric values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Train an SVM Classifier
svm_model = SVC(kernel='linear', C=1.0)  # Linear SVM
svm_model.fit(X_train, y_train_encoded)

# Make Predictions
y_pred = svm_model.predict(X_test)

# Evaluate Model
accuracy = accuracy_score(y_test_encoded, y_pred)
print(f"SVM Model Accuracy: {accuracy:.4f}")
precision = precision_score(y_test_encoded, y_pred, average='weighted')
print(f"SVM Model Precision: {precision:.4f}")
recall = recall_score(y_test_encoded, y_pred, average='weighted')
print(f"SVM Model Recall Score: {recall:.4f}")
f1 = f1_score(y_test_encoded, y_pred, average='weighted')
print(f"SVM Model F1 Score: {recall:.4f}")






Extraction complete.


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


SVM Model Accuracy: 0.4445
SVM Model Precision: 0.4514
SVM Model Recall Score: 0.4445
SVM Model F1 Score: 0.4445
